### **Homework: Introduction**

**Q1: Running ElasticSearch**

Run elastic search using `curl localhost:9200` and get the cluster information. What's the `version.build_hash` value?

*"42f05b9372a9a4a470db3b52817899b99a76ee73"*

In [1]:
!curl localhost:9200

{
  "name" : "95fa37dcd229",
  "cluster_name" : "docker-cluster",
  "cluster_uuid" : "JdWucIFvTIS93LH3YjpSrw",
  "version" : {
    "number" : "8.4.3",
    "build_flavor" : "default",
    "build_type" : "docker",
    "build_hash" : "42f05b9372a9a4a470db3b52817899b99a76ee73",
    "build_date" : "2022-10-04T07:17:24.662462378Z",
    "build_snapshot" : false,
    "lucene_version" : "9.3.0",
    "minimum_wire_compatibility_version" : "7.17.0",
    "minimum_index_compatibility_version" : "7.0.0"
  },
  "tagline" : "You Know, for Search"
}


**Q2: Indexing the Data**

Load, prepare and index the FAQ data as per the course videos. What function do you use to add your data to ElasticSearch?

*index*

In [3]:
import requests
from elasticsearch import Elasticsearch
from tqdm.auto import tqdm

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"}
        }
    }
}

index_name = "course-questions"
es = Elasticsearch("http://localhost:9200")

# create and index the documents only once
try:
    response = es.indices.create(index=index_name, body=index_settings)

    for doc in tqdm(documents):
        es.index(index=index_name, document=doc)
except:
    pass

/home/codespace/.local/share/virtualenvs/0_RAG_Workshop-Y-8dc-rB/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


**Q3: Searching**

Execute the query: *How do I execute a command in a running docker container?* Use only the `question` and `text` fields, give `question` a boost of 4, and use `"type": "best_fields"`. What's the score for the top-ranking result?

*84.05*

In [21]:
user_question = "How do I execute a command in a running docker container?"

search_query = {
    "size": 1,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": user_question,
                    "fields": ["question^4", "text"],
                    "type": "best_fields"
                }
            },
        }
    }
}

response = es.search(index=index_name, body=search_query)

for hit in response['hits']['hits']:
    doc = hit['_source']
    print(f"Course: {doc['course']}")
    print(f"Question: {doc['question']}")
    print(f"Answer: {doc['text']}...")
    print(f"Score: {hit['_score']}\n")


Course: machine-learning-zoomcamp
Question: How do I debug a docker container?
Answer: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.
docker run -it --entrypoint bash <image>
If the container is already running, execute a command in the specific container:
docker ps (find the container-id)
docker exec -it <container-id> bash
(Marcos MJD)...
Score: 84.050095



**Q4: Filtering**

Limit the results to the top 3 from the *machine learning zoomcamp* course. What's the 3rd question returned?

*How do I copy files from a different folder into docker container's working directory?*

In [24]:
search_query = {
    "size": 3,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": user_question,
                    "fields": ["question^4", "text"],
                    "type": "best_fields"
                }
            },
                        # search only the data engineering course documents
            "filter": {
                "term": {
                    "course": "machine-learning-zoomcamp"
                }
            }
        }
    }
}

response = es.search(index=index_name, body=search_query)

for hit in response['hits']['hits']:
    doc = hit['_source']
    print(f"Course: {doc['course']}")
    print(f"Question: {doc['question']}")
    print(f"Answer: {doc['text'][:100]}...\n")

Course: machine-learning-zoomcamp
Question: How do I debug a docker container?
Answer: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a ba...

Course: machine-learning-zoomcamp
Question: How do I copy files from my local machine to docker container?
Answer: You can copy files from your local machine into a Docker container using the docker cp command. Here...

Course: machine-learning-zoomcamp
Question: How do I copy files from a different folder into docker container’s working directory?
Answer: You can copy files from your local machine into a Docker container using the docker cp command. Here...



**Q5: Building a Prompt**

Use the records returned from ElasticSearch in the previous question as the template to build the context. Separate context entries by a double-linebreak (`\n\n`). Then use that context along with the query from question 3 to construct a prompt. What's the length of that prompt?

**

In [37]:
context_template = """
Question: {question}
Answer: {text}
""".strip()

context_docs = response.copy()

context_result = ""
for doc in context_docs:
    doc_str = context_template.format(**doc)

TypeError: str.format() argument after ** must be a mapping, not str

In [28]:
context_result = ""
for doc in context_docs:
    doc_str = context_template.format(doc)
    context_result += ("\n\n" + doc_str)

context = context_result.strip()
print(context)

prompt = f"""
You're a course teaching assistant.
Answer the user QUESTION based on the CONTEXT from the FAQ database. 
Only use the facts from the CONTEXT when answering the QUESTION.

QUESTION: {user_question}

CONTEXT:

{context}
""".strip()


KeyError: 'user_question'